In [1]:
import re

In [2]:
def parser(file_path='Quake.txt'):
    """ Essa função recebe o caminho ao arquivo a ser utilizado, neste caso, quake.txt, e retorna as informações
        pedidas no desáfio em uma lista de dicionários em ordem de jogo
    """
    file = open('Quake.txt', 'r')

    time_log = r"\s*\d+:\d+\s"

    initgame = re.compile(time_log+"initgame", flags=re.IGNORECASE)

    connect = re.compile(time_log+"clientconnect", re.IGNORECASE)

    info_change = re.compile(time_log+"ClientUserinfoChanged", re.IGNORECASE)

    kill = re.compile(time_log+"Kill", re.I)

    shutdown = re.compile(time_log+"shutdowngame", re.I)

    result = []
    game = 1
    for i in file:
        if initgame.match(i):                                                           ## Procura inicio da partída
            result.append({'game': game, 'status':{'total_kills': 0, 'players': []}}) 
            ids = []

        elif connect.match(i):                                                          ## '' novo jogador conectado, define ids
            player_id = i.split()[2]
            if player_id not in ids:
                result[game-1]['status']['players'].append({'id': player_id, 'name':"", 'kills': 0, "old_names":[]})
                ids.append(player_id)
            
            
        elif info_change.match(i):                                                      ##  '' confere se o nome foi alterado, guarda nome substituido | define nome
            player_id = i.split()[2]
            name = i.split('\\')[1]
            ids_len = len(ids)       

            for i in range(ids_len):
                if result[game-1]['status']['players'][i]['id'] == player_id:
                    if result[game-1]['status']['players'][i]['name'] == name:
                        break

                    elif result[game-1]['status']['players'][i]['name'] == "":
                        result[game-1]['status']['players'][i]['name'] = name
                        
                    else:
                        result[game-1]['status']['players'][i]['old_names'].append(result[game-1]['status']['players'][i]['name'])
                            
                        result[game-1]['status']['players'][i]['name'] = name
                    break

        
        elif kill.match(i):                                      ## adiciona ao total de kills, adiciona kills a quem mata, e diminui kills a quem morre pelo mundo
        
            result[game-1]['status']['total_kills'] +=1
            
            ids_len = len(ids)
            line = i.split(': ')[1].split()                             ## [0] quem matou [1] quem morreu

            if line[0] == 1022:             # 1022 world id 
                killed_id = line[1]

                for k in range(ids_len):
                    
                    if result[game-1]['status']['players'][k]['id'] == killed_id :
                        result[game-1]['status']['players'][k]['kills'] -= 1
                        break
            else :
                killer_id = line[0]
                for k in range(ids_len):
                    
                    if result[game-1]['status']['players'][k]['id'] == killer_id :
                        result[game-1]['status']['players'][k]['kills'] += 1
                        break
            
            
        elif shutdown.match(i):
            game +=1
    
    file.close()

    return result


In [3]:
games = parser()

Para importar para um arquivo em JSON, basta escolher o game, e o nome do arquivo desejado, lembrando que o indice para o primeiro game é 0.

In [5]:
import json

with open('game_1.json', 'w') as json_file:
  json.dump(games[0], json_file, indent=2, sort_keys=True)